# Agent - 基本的なAPIを組み合わせて作成

### Agent内容
- 質問を受け取る
- 質問に関係のある内部文書を検索し、回答する
- 内部文書を基にして回答ができなければ、外部データを検索し、質問に回答する

pip install google-genai chromadb tavily
pip install langchain langchain-core langchain-community langchain-text-splitters

In [10]:
from pprint import pprint
import os, re, glob

import google.genai as genai
import chromadb
import tavily 
import langchain_core
from langchain_core.documents import Document
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

print(f"genai={genai.__version__}, chromadb={chromadb.__version__}, " + 
      f"langchain_core={langchain_core.__version__}, ")

# Geminiクライアントを作成
with open('../keys/Google_API_KEY.txt', 'r') as f:  # ファイルからアクセスキーを取得
    api_key = f.read().strip()
GEMINI_CLIENT = genai.Client(api_key=api_key)
# Geminiモデルを指定
GEMINI_EMBEDDING_MODEL = 'gemini-embedding-001'
#GEMINI_LLM_MODEL = 'gemini-2.5-flash'
GEMINI_LLM_MODEL = 'gemini-3-pro-preview'

# Chromaクライアントを作成
CHROMA_CLIENT = chromadb.EphemeralClient()  # インメモリで作成

# Tavilyクライアントを作成: Agentから、ネット検索
with open('../keys/Tavily_API_KEY.txt', 'r') as f:  # ファイルからアクセスキーを取得
    api_key = f.read().strip()
TAVILY_CLIENT = tavily.TavilyClient(api_key=api_key)

genai=1.53.0, chromadb=1.3.5, langchain_core=1.1.1, 


## 内部文書の読み込み、VectorStoreへ登録

In [ ]:
# ローカルデータの読み込み、VectorStoreの作成

# テキストファイル・メタデータ
matadata_dic = {
    '2_chumonno_oi_ryoriten.txt': {
        'genre': 'novel', 'title':'注文の多い料理店',  'author':'宮沢賢治',   'year':'1924'},
    '2_kazeno_matasaburo.txt': {
        'genre': 'novel', 'title':'風の又三郎',        'author':'宮沢賢治',   'year':'1934'},
    '2_serohikino_goshu.txt': {
        'genre': 'novel', 'title':'セロ弾きのゴーシュ', 'author':'宮沢賢治',   'year':'1934'},
    '2_gingatetsudono_yoru.txt': {
        'genre': 'novel', 'title':'銀河鉄道の夜',      'author':'宮沢賢治',   'year':'1934'},
    '3_bocchan.txt': {
        'genre': 'novel', 'title':'坊ちゃん',         'author':'夏目漱石',   'year':'1906'},
    '4_wagahaiwa_nekodearu.txt': {
        'genre': 'novel', 'title':'吾輩は猫である',    'author':'夏目漱石',   'year':'1905'},
    '4_kaijin_nijumenso.txt': {
        'genre': 'novel', 'title':'怪人二十面相',      'author':'江戸川乱歩', 'year':'1936'},
    '5_sanshodayu.txt': {
        'genre': 'novel', 'title':'山椒大夫',         'author':'森鷗外',     'year':'1915'},
}
null_dic = {'genre':'','title':'', 'author':'','year':''}

# 小説データをロード
novels_dir = '../novels/'
loader = DirectoryLoader(novels_dir, glob='*.txt') # ディレクトリ内の.txtを指定して
documents = loader.load()   # ドキュメント(メタデータ(ファイル名)+テキスト)としてロード

# 各ドキュメントにメタデータを付与
for doc in documents:
    fn = os.path.basename(doc.metadata['source'])
    doc.metadata.update({'filename': fn} | matadata_dic.get(fn, null_dic))

# ドキュメントをチャンクへ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=100,
    separators=['。', '\n'] # 分割位置の指定
)
chunks = text_splitter.split_documents(documents)
print(f"作成されたChunk数={len(chunks)}")

# Embedding取得
response = GEMINI_CLIENT.models.embed_content(
    model=GEMINI_EMBEDDING_MODEL,
    contents=[c.page_content for c in chunks]
)
doc_emb = [e.values for e in response.embeddings]
 
# ChromaDB: collectionの初期化
collection_name = 'Internal_Doc'
try:
    CHROMA_CLIENT.delete_collection(collection_name)
except:
    pass
collection = CHROMA_CLIENT.create_collection(
    name = collection_name,
    metadata={'hnsw:space': 'cosine'}  # 距離メトリック = 'cosine'
)

# ChromaDBへ一括で追加
collection.add(
    ids=[str(i) for i in range(len(chunks))], # id=Chunk連番
    embeddings=doc_emb,
    documents=[c.page_content for c in chunks],
    metadatas=[c.metadata     for c in chunks]
)
print(f"DBに追加されたベクトル数: {collection.count()}")

作成されたChunk数=97
DBに追加されたベクトル数: 97


## Agentから利用するツールの定義

In [12]:
# Agent

# ネット情報の検索
def tavily_search(query, max_results=5, time_range=None):
    resp = TAVILY_CLIENT.search(
        query=query,
        max_results=max_results,
        time_range=time_range
    )
    # content のみ抽出し連結
    results_text = "\n\n".join(
        [f"{r['title']} ({r['url']})\n{r['content']}" for r in resp.get("results", [])]
    )
    return results_text

# 内部ファイルの検索
def retriever(query, k=3):
    query_emb = GEMINI_CLIENT.models.embed_content(
        model=GEMINI_EMBEDDING_MODEL,
        contents=query
    ).embeddings[0].values
    # ChromaDBで類似検索（＝retrieval）=======
    results = collection.query(
        query_embeddings=[query_emb],
        n_results = 3,
        include = ['documents', 'metadatas'],
    )
    return results

## Agentの実行

In [ ]:
# エージェントに回答文を生成させる、　注：蜘蛛の糸はデータに含まれていない
query = '御釈迦様は蜘蛛の糸を何に使いましたか'
print(f"質問: {query}\n")

# 内部文書を検索
retrieved_texts = retriever(query)['documents']
#print(len(retrieved_texts['documents']), retrieved_texts['documents'])

# 内部文書を元にqueryを実行
prompt = f'''
後述の参照文書を参照して、質問に根拠付きで回答してください。
情報が見つからない場合には「わかりません」と回答してください。

質問: {query}

参照文書:
''' + '\n'.join(retrieved_texts[0])

response = GEMINI_CLIENT.models.generate_content(
    model=GEMINI_LLM_MODEL,
    contents=[prompt]
)
print(response.text)

# 内部文書を元にしたqueryへの回答を評価
prompt = f'''
後述の出力文を参照して、
質問に対して情報を見つけてかつ内容のある回答ができていれば「YES」で、
質問に対しての情報を見つけられない、または、回答が十分で無ければ「NO」で
回答してください。

質問: {query}

出力文:
''' + response.text

response = GEMINI_CLIENT.models.generate_content(
    model=GEMINI_LLM_MODEL,
    contents=[prompt]
)
print(f"\n回答の判断: {response.text}\n")

if response.text != 'YES':
    results_text = tavily_search(query)
    prompt = f'''
後述の外部参照文書を参照して、質問に根拠付きで回答してください。

質問: {query}

外部検索文書:
''' + '\n'.join(results_text)
    
    response = GEMINI_CLIENT.models.generate_content(
        model=GEMINI_LLM_MODEL,
        contents=[prompt]
    )
    print(response.text)

質問: 御釈迦様は蜘蛛の糸を何に使いましたか

わかりません。

（参照文書には「山椒大夫」と「怪人二十面相（少年探偵団）」に関する記述が含まれていますが、御釈迦様や蜘蛛の糸に関する記述は見当たりません。）

回答の判断: NO

外部参照文書に基づき、御釈迦様が蜘蛛の糸を何に使ったかについて回答します。

**回答：**
御釈迦様は、地獄の底にいる罪人カンダタ（ガンダタ）を**救い出すための手段として**、極楽から地獄へ向けて蜘蛛の糸を垂らすことに使いました。

**根拠：**

1.  **善行への報いとして救済するため**
    御釈迦様は地獄の様子をご覧になり、カンダタが過去に一度だけ蜘蛛を助けた善行を思い出しました。その「報い」として、彼を地獄から救い出そうとされました。
    > 「このガンダタには蜘蛛を助けた事があるのを御思い出しになりました。そうしてそれだけの善い事をした 報 むくい」（参照：Jun Ashikari『蜘蛛の糸』について）

2.  **物理的に地獄へ下ろす道具として**
    御釈迦様は蜘蛛の糸を手に取り、極楽の蓮の間から地獄の底へ向かってまっすぐに下ろしました。
    > 「御釋迦様はその蜘蛛の糸をそっとお手に御取りになって, 玉のやうな白蓮の間から, 遥か下にある地獄の底へ, まっすぐに. それを御下しなさいました」（参照：[PDF] 芥川龍之介が不用意に扱った素材）

3.  **「教え」や「ヒント」を与えるため（解釈）**
    一部の解釈として、直接引き上げるのではなく、カンダタに地獄から抜け出すための自覚や理解を促す「間接的なヒント」を与えるために使われたとも説明されています。
    > 「お釈迦様はカンダタに、地獄から抜け出すために必要なことは何か、を理解する間接的なヒントを与えました。それが蜘蛛の糸です。」（参照：Yahoo!知恵袋 ベストアンサー）
